# VENTAS

In [1]:
from datetime import date, timedelta, datetime
import pandas as pd
import numpy as np

# Mostrar todo el contenido de las celdas (sin recortes)
pd.set_option('display.max_colwidth', None)

# Mostrar todas las columnas 
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_parquet("/home/donsson/proyectos/VENTAS/ventashistoricas100semanas.parquet")

In [3]:
df_r = df[(df["date_invoice"]>="2024-11-18") & (df["date_invoice"]<="2025-11-19")]

In [4]:
df_r["date_invoice"].min()

df_r["store_name"].unique()

array(['SUCURSAL VALLADOLID', 'SUCURSAL BUCARAMANGA', 'SUCURSAL CALLE 6',
       'SUCURSAL MEDELLIN', 'PRINCIPAL COTA', 'SUCURSAL NORTE',
       'SUCURSAL BARRANQUILLA', 'SUCURSAL CALI'], dtype=object)

In [5]:
df_r = df_r.drop(columns=["origin","id_x","id_y","partner_id_num","product_id"])


In [6]:
import re

df_r["product_ref"] = df_r["product_name"].str.extract(r"\[([^\]]+)\]")

df_r["date_invoice"] = pd.to_datetime(df_r["date_invoice"], errors="coerce")
df_r["Año"] = df_r["date_invoice"].dt.year
df_r["Mes"] = df_r["date_invoice"].dt.month


In [7]:
df_r["periodo"] = df_r["Año"].astype(str) + "-" + df_r["Mes"].astype(str).str.zfill(2)


df_pivot = df_r.pivot_table(
    index=["store_name", "product_ref"],
    columns="periodo",
    values="quantity",
    aggfunc="sum",
    fill_value=0
).reset_index()

df_pivot = df_pivot.rename(columns={"store_name": "Sucursal"})


df_pivot.sample(10)


periodo,Sucursal,product_ref,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11
7301,SUCURSAL CALLE 6,DAE03197025,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8899,SUCURSAL MEDELLIN,DAR02176025,1.0,3.0,3.0,3.0,0.0,0.0,2.0,1.0,2.0,1.0,2.0,2.0,0.0
2389,SUCURSAL BARRANQUILLA,BCS00501125,0.0,1.0,4.0,0.0,6.0,3.0,1.0,0.0,2.0,9.0,0.0,1.0,0.0
2064,PRINCIPAL COTA,DLE01003136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5149,SUCURSAL CALI,BCS00352125,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0
5001,SUCURSAL CALI,BAE08201125,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5245,SUCURSAL CALI,BCS10499125,14.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0
6474,SUCURSAL CALLE 6,BCS00270125,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,4.0,1.0,3.0,2.0
8363,SUCURSAL MEDELLIN,BLS00234125,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5604,SUCURSAL CALI,DAB02976025,0.0,0.0,6.0,7.0,12.0,2.0,0.0,3.0,10.0,6.0,3.0,6.0,3.0


In [8]:
df_pivot["Sucursal"].unique()

array(['PRINCIPAL COTA', 'SUCURSAL BARRANQUILLA', 'SUCURSAL BUCARAMANGA',
       'SUCURSAL CALI', 'SUCURSAL CALLE 6', 'SUCURSAL MEDELLIN',
       'SUCURSAL NORTE', 'SUCURSAL VALLADOLID'], dtype=object)

# INVENTARIO

In [9]:
inv = pd.read_excel("/home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx")
inv["product_ref"] = inv["Complete name"].str.extract(r"\[([A-Z0-9]+)\]")

In [10]:
inv.head()

,Sucursal,Estado,Codigo_interno,Complete name,Clase,Marca,categoria_producto,Cantidad,Costo unitario,Costo total,product_ref
0,PRINCIPAL COTA,Existencias,004 DA4207,[DAE04207004] DA4207 FILTRO AIRE SEGURIDAD KOMATSU (004 AF574SY),C,AUT*PARTS,Filtro,1.0,1.00,1.00,DAE04207004
1,PRINCIPAL COTA,Existencias,004 G314,[DLE00314004] G314 FILTRO ACEITE ESPECIAL (004 F62H),C,AUT*PARTS,Filtro,2.0,1890.00,3780.00,DLE00314004
2,PRINCIPAL COTA,Existencias,004 G401,[DLE00401004] G401 FILTRO ACEITE FIAT-ALLIS 75215486 (004 F75215486),C,AUT*PARTS,Filtro,7.0,1.00,7.00,DLE00401004
3,SUCURSAL MEDELLIN,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,2.0,1694.96,3389.92,DAE05128004
4,PRINCIPAL COTA,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,1.0,1694.96,1694.96,DAE05128004


In [11]:
inv["Sucursal"].unique()

array(['PRINCIPAL COTA', 'SUCURSAL MEDELLIN', 'SUCURSAL BUCARAMANGA',
       'SUCURSAL NORTE', 'SUCURSAL CALLE 6', 'SUCURSAL VALLADOLID',
       'SUCURSAL BARRANQUILLA', 'MOSTRADOR COTA', 'SUCURSAL CALI',
       'CUMMINS DE LOS ANDES', 'CUMMINS DE LOS ANDES MEDELLIN',
       'ANTIOQUEÑA DE LUBRICANTES SGP SAS', 'EXP. ECUADOR',
       'BODEGA VENDEDOR WILMER GIL', 'EXP.REP.DOMINICANA', nan,
       'PRESTAMOS INTERNOS', 'FILTRO EN REPROCESO', 'CSS CONSTRUCTORES',
       'STOCK SIGMA ENERGY', 'MATERIA PRIMA', 'CEMEX BOSA',
       'CEMEX CENTENARIO'], dtype=object)

# DESPACHOS

In [12]:
des = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/despachos.csv")

In [13]:
d = des[des["semana"]==46]

d = d[["store_name","product_ref","semana","Costo unitario","Clasificacion","inventario_ideal"]]


# DF PRINCIPAL COTA

In [14]:
df_bod1 = inv[inv["Sucursal"] == "PRINCIPAL COTA"][[
    "product_ref", "Cantidad"
]].rename(columns={"Cantidad": "inv_bodega_1"})

df_bod1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2925 entries, 0 to 15267
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_ref   2925 non-null   object 
 1   inv_bodega_1  2925 non-null   float64
dtypes: float64(1), object(1)
memory usage: 68.6+ KB


# UNIR

In [15]:
import pandas as pd
import xmlrpc.client

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","semana","clasificacion"]}
)

cl_org = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

cl_org.rename(columns={"product_name": "product_ref"}, inplace=True)


cl_org["store_id_num"] = cl_org["sucursal_id"].apply(extract_id)
cl_org["Sucursal"]   = cl_org["sucursal_id"].apply(extract_name)


In [16]:
cl = cl_org[["Sucursal","product_ref","semana","clasificacion"]] 

cl.sample(10)

,Sucursal,product_ref,semana,clasificacion
1675,SUCURSAL BARRANQUILLA,DAB08227025,46,C
32307,SUCURSAL VALLADOLID,BHE00283125,46,C
26624,SUCURSAL CALLE 6,DAB09069025,46,C
33670,SUCURSAL VALLADOLID,DAE02629025,46,C
6067,SUCURSAL MEDELLIN,BHE00950125,46,C
17642,SUCURSAL CALI,BAB08087125,46,C
18430,SUCURSAL CALI,BCS00544125,46,C
35534,SUCURSAL BUCARAMANGA,DLE01030189,46,C
16232,SUCURSAL CALI,DCS00242189,46,C
30298,SUCURSAL VALLADOLID,DCS00780125,46,C


In [17]:
# Normalizar nombres
d = d.rename(columns={
    "store_name": "Sucursal",
    "Clasificacion":"clase mia"
})

# Asegurar que product_ref en ambos DF es string
d["product_ref"] = d["product_ref"].astype(str).str.strip()
inv["product_ref"] = inv["product_ref"].astype(str).str.strip()


In [18]:
d = d.merge(df_bod1, on="product_ref", how="left")

d = d.merge(cl,on=["Sucursal","product_ref","semana"],how="left")


In [19]:
inv_suc = inv[[
    "Sucursal", "product_ref","Cantidad"
]]

inv_suc = inv_suc.rename(columns={
    "Cantidad": "inv_actual",
})

d = d.merge(
    inv_suc,
    on=["Sucursal", "product_ref"],
    how="left"
)

d = d.fillna(0)


In [20]:
d.sample(10)

,Sucursal,product_ref,semana,Costo unitario,clase mia,inventario_ideal,inv_bodega_1,clasificacion,inv_actual
9765,SUCURSAL BUCARAMANGA,BCS00573125,46,52059.07,B,22.62,19.0,A,4.0
20783,SUCURSAL CALLE 6,DAP08203025,46,36999.87,C,0.01,0.0,C,0.0
15639,SUCURSAL CALI,DAC00082189,46,7991.30,C,0.00,10.0,C,0.0
11633,SUCURSAL BUCARAMANGA,DAE03754025,46,13169.00,C,0.00,0.0,C,0.0
5183,SUCURSAL BARRANQUILLA,BCS00299125,46,16772.31,C,39.95,191.0,A,18.0
38469,SUCURSAL VALLADOLID,DCE00490136,46,6932.47,C,0.00,3.0,C,0.0
6978,SUCURSAL BARRANQUILLA,DAE01026115,46,5842.41,C,0.00,0.0,C,0.0
1394,MOSTRADOR COTA,BHS00415125,46,128069.06,C,0.00,4.0,0,0.0
35808,SUCURSAL VALLADOLID,BCS00352125,46,71368.96,C,0.00,64.0,C,0.0
21922,SUCURSAL MEDELLIN,AGBGRASA030,46,26.84,C,0.00,0.0,C,0.0


In [21]:
df_d = d.merge(
    df_pivot,
    on=["Sucursal", "product_ref"],
    how="left"
).fillna(0)


In [22]:
df_d["inventario_ideal"] = df_d["inventario_ideal"].round(0)

df_d["diferencia"] = df_d["inventario_ideal"] - df_d["inv_actual"]


# --- Identificar las columnas de meses ---
cols_meses = [
    "2024-11","2024-12","2025-01","2025-02","2025-03","2025-04",
    "2025-05","2025-06","2025-07","2025-08","2025-09","2025-10","2025-11"
]

# ⚠️ Son 13 columnas porque incluye 2024-11 y 2025-11
# Pero tú dijiste que quieres el promedio SOLO de 12 meses.
# Así que eliminamos uno: normalmente el último está incompleto (2025-11)
df_d[cols_meses] = df_d[cols_meses].apply(pd.to_numeric, errors="coerce")
df_d["promedio_12m"] = df_d[cols_meses].sum(axis=1) / 12


df_d["max_12m"] = df_d[cols_meses].max(axis=1)
df_d["meses_con_ventas"] = (df_d[cols_meses] > 0).sum(axis=1)


# Eliminar filas donde Sucursal sea 'MOSTRADOR COTA'
df_d = df_d[~df_d["Sucursal"].str.upper().eq("MOSTRADOR COTA")].copy()


In [23]:
df_d = df_d[df_d["diferencia"]<0]

filtros_aceites = df_d["product_ref"].str.startswith(("ALG", "ALT","AGB"), na=False)

# Eliminar esas filas del df principal
df_d = df_d[~filtros_aceites].copy()

df_d.info()

df_d.sample(10)

<class 'pandas.core.frame.DataFrame'>
Index: 8659 entries, 4298 to 39189
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Sucursal          8659 non-null   object 
 1   product_ref       8659 non-null   object 
 2   semana            8659 non-null   int64  
 3   Costo unitario    8659 non-null   float64
 4   clase mia         8659 non-null   object 
 5   inventario_ideal  8659 non-null   float64
 6   inv_bodega_1      8659 non-null   float64
 7   clasificacion     8659 non-null   object 
 8   inv_actual        8659 non-null   float64
 9   2024-11           8659 non-null   float64
 10  2024-12           8659 non-null   float64
 11  2025-01           8659 non-null   float64
 12  2025-02           8659 non-null   float64
 13  2025-03           8659 non-null   float64
 14  2025-04           8659 non-null   float64
 15  2025-05           8659 non-null   float64
 16  2025-06           8659 non-null   float64
 

,Sucursal,product_ref,semana,Costo unitario,clase mia,inventario_ideal,inv_bodega_1,clasificacion,inv_actual,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,diferencia,promedio_12m,max_12m,meses_con_ventas
16077,SUCURSAL CALI,DAE04407025,46,29339.41,C,2.0,5.0,C,8.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.416667,2.0,4
23184,SUCURSAL MEDELLIN,BAS07831125,46,191908.00,C,0.0,0.0,C,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-24.0,0.000000,0.0,0
26545,SUCURSAL MEDELLIN,DAB09270025,46,15556.78,C,2.0,42.0,C,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,-12.0,1.000000,12.0,1
11977,SUCURSAL BUCARAMANGA,DAE09147025,46,32169.06,C,0.0,6.0,C,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-2.0,0.083333,1.0,1
25393,SUCURSAL MEDELLIN,BLS00308125,46,237135.80,C,0.0,0.0,C,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.083333,1.0,1
21385,SUCURSAL CALLE 6,DCE10129138,46,24590.14,C,13.0,497.0,A,16.0,9.0,4.0,7.0,6.0,6.0,16.0,11.0,12.0,11.0,3.0,4.0,8.0,13.0,-3.0,9.166667,16.0,13
25973,SUCURSAL MEDELLIN,DAB02969025,46,40993.72,C,0.0,22.0,C,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.000000,0.0,0
29589,SUCURSAL MEDELLIN,DCE01019189,46,18924.00,C,0.0,191.0,C,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.000000,0.0,0
24238,SUCURSAL MEDELLIN,BHE00009125,46,26033.73,C,25.0,218.0,C,51.0,0.0,9.0,2.0,4.0,62.0,68.0,2.0,20.0,19.0,0.0,40.0,16.0,0.0,-26.0,20.166667,68.0,10
21582,SUCURSAL CALLE 6,DCX00053137,46,711905.77,B,2.0,0.0,A,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,0.0,1.0,1.0,-2.0,0.833333,4.0,5


In [24]:
#df_d.to_csv("/home/donsson/proyectos/DEVOLUCIONES MERCANCIA/devoluciones.csv")

## SE DEVUELVE SI NO SE A VENDIDO EN UN AÑO

In [25]:
# 1️⃣ Crear df devoluciones si no existe
devoluciones = pd.DataFrame()

# 2️⃣ Filtrar los que tienen promedio de 12 meses igual a cero
df_prom0 = df_d[df_d["promedio_12m"] == 0].copy()

# 3️⃣ Crear la columna unidades_a_devolver = todo el inventario actual
df_prom0["unidades_a_devolver"] = df_prom0["inv_actual"]

# 4️⃣ Pasar estas filas al dataframe devoluciones
devoluciones = pd.concat([devoluciones, df_prom0], ignore_index=True)

# 5️⃣ Eliminar estas filas de df_d (porque ya fueron procesadas)
df_d = df_d[df_d["promedio_12m"] != 0].copy()


### LOS AAA SE QUEDAN

In [26]:
# 1) Filtrar los AAA (estos se quedan)
df_se_quedan = df_d[df_d["clase mia"] == "AAA"].copy()

# 2) Removerlos de df_d para no procesarlos en devoluciones
df_d = df_d[df_d["clase mia"] != "AAA"].copy()


In [27]:
# 1) Filtrar solo AAA
df_AAA = df_d[df_d["clasificacion"] == "AAA"].copy()

# 2) Calcular inventario para 4 meses
df_AAA["inv_4_meses"] = df_AAA["promedio_12m"] * 4

# 3) Calcular unidades que sobran
df_AAA["unidades_devolver"] = df_AAA["inv_actual"] - df_AAA["inv_4_meses"]

# 4) Filtrar productos que realmente se deben devolver
df_AAA_devolver = df_AAA[
    (df_AAA["unidades_devolver"] > 0) &                      # Sobra inventario
    (df_AAA["inv_bodega_1"] < df_AAA["inv_actual"]) &        # Bodega tiene menos que sucursal
    (df_AAA["inv_bodega_1"] < 2 * df_AAA["unidades_devolver"])  # Nueva condición
].copy()

# 5) Redondear unidades a devolver
df_AAA_devolver["unidades_devolver"] = df_AAA_devolver["unidades_devolver"].round(0)

# 6) Añadir a devoluciones
devoluciones = pd.concat([devoluciones, df_AAA_devolver], ignore_index=True)

# 7) Restar unidades devueltas del df principal
df_d.loc[df_AAA_devolver.index, "inv_actual"] -= df_AAA_devolver["unidades_devolver"]

# 8) Remover AAA ya procesados del df principal
df_d = df_d[df_d["clasificacion"] != "AAA"].copy()


## A clase en las dos clasificaciones

In [28]:
# Filtrar solo los productos que cumplen A / A
df_A = df_d[(df_d["clase mia"] == "A") & (df_d["clasificacion"] == "A")].copy()

def unidades_a_devolver(row):
    prom = row["promedio_12m"]
    actual = row["inv_actual"]
    meses = row["meses_con_ventas"]
    b1 = row["inv_bodega_1"]

    # 1️⃣ INVENTARIO A CONSERVAR
    if meses >= 12:
        inv_seguro = prom * 6
    elif 9 <= meses <= 11:
        inv_seguro = prom * 4
    elif 6 <= meses <= 8:
        inv_seguro = prom * 2
    else:
        inv_seguro = prom * 1

    # 2️⃣ Sobrante
    unidades_sobra = actual - inv_seguro

    if unidades_sobra <= 0:
        return 0.0

    # 3️⃣ Regla bodega
    if b1 < prom * 2:
        return 0.0

    # 🔥 FORZAR ESCALAR
    return float(round(unidades_sobra))

# 0️⃣ Arreglar si quedó basura en la columna
if "unidades_devolver" in df_A.columns:
    df_A = df_A.drop(columns=["unidades_devolver"])

# 1️⃣ Calcular unidades a devolver
df_A["unidades_devolver"] = df_A.apply(unidades_a_devolver, axis=1)

# 2️⃣ Filtrar solo los que se devuelven
df_A_devolver = df_A[df_A["unidades_devolver"] > 0].copy()

# 3️⃣ Agregar a devoluciones
devoluciones = pd.concat([devoluciones, df_A_devolver], ignore_index=True)

# 4️⃣ Restar el inventario en df_d
df_d.loc[df_A_devolver.index, "inv_actual"] -= df_A_devolver["unidades_devolver"]

# 5️⃣ Quitar los A del dataframe original
df_d = df_d[df_d["clase mia"] != "A"].copy()



### SOLO A EN MI CALSIFICACION DEVUELVE CUANDO HAYAN MENOS DE 15 EN BD 1 LA CANTIDAD DE LA DIFERENCIA CON EL INV IDEAL 

In [34]:
# Regla: clase mia = A y inv_bodega_1 < 10

filtro_A_bodega_baja = (df_d["clase mia"] == "A") & (df_d["inv_bodega_1"] < 10)

# Extraer a un df temporal
df_A_bodega_baja = df_d[filtro_A_bodega_baja].copy()

# Definir unidades a devolver = ABS(diferencia)
df_A_bodega_baja["unidades_devolver"] = df_A_bodega_baja["diferencia"].abs()

# Agregar a df_devoluciones
devoluciones = pd.concat([devoluciones, df_A_bodega_baja], ignore_index=True)

# Eliminar del df_d original
df_d = df_d[~filtro_A_bodega_baja].copy()

In [30]:
df_d.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7018 entries, 4298 to 39189
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Sucursal          7018 non-null   object 
 1   product_ref       7018 non-null   object 
 2   semana            7018 non-null   int64  
 3   Costo unitario    7018 non-null   float64
 4   clase mia         7018 non-null   object 
 5   inventario_ideal  7018 non-null   float64
 6   inv_bodega_1      7018 non-null   float64
 7   clasificacion     7018 non-null   object 
 8   inv_actual        7018 non-null   float64
 9   2024-11           7018 non-null   float64
 10  2024-12           7018 non-null   float64
 11  2025-01           7018 non-null   float64
 12  2025-02           7018 non-null   float64
 13  2025-03           7018 non-null   float64
 14  2025-04           7018 non-null   float64
 15  2025-05           7018 non-null   float64
 16  2025-06           7018 non-null   float64
 

In [31]:
df_d.to_excel("/home/donsson/proyectos/DEVOLUCIONES MERCANCIA/devoluciones.xlsx")

In [32]:
devoluciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1517 entries, 0 to 1516
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Sucursal             1517 non-null   object 
 1   product_ref          1517 non-null   object 
 2   semana               1517 non-null   int64  
 3   Costo unitario       1517 non-null   float64
 4   clase mia            1517 non-null   object 
 5   inventario_ideal     1517 non-null   float64
 6   inv_bodega_1         1517 non-null   float64
 7   clasificacion        1517 non-null   object 
 8   inv_actual           1517 non-null   float64
 9   2024-11              1517 non-null   float64
 10  2024-12              1517 non-null   float64
 11  2025-01              1517 non-null   float64
 12  2025-02              1517 non-null   float64
 13  2025-03              1517 non-null   float64
 14  2025-04              1517 non-null   float64
 15  2025-05              1517 non-null   f

In [33]:
devoluciones.to_excel("/home/donsson/proyectos/DEVOLUCIONES MERCANCIA/devoluciones_fijas.xlsx")